## Web Scraping for Celestial Coordinates of Planets from TheSkyLive.com

In [32]:
import bs4
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as soup
import time

In [2]:
class CelestialBody:
    lastUpdate = time.localtime()
    
    def __init__(self, name, ra, dec):
        self.name = name
        self.ra = ra
        self.dec = dec
        
    def displayInfo(self):
        return "Name: " + self.name + " R.A: " + self.ra + " Dec: " + self.dec
    
    @classmethod
    def displayLastUpdate(CelestialBody):
        month = CelestialBody.lastUpdate.tm_mon
        day = CelestialBody.lastUpdate.tm_mday
        year = CelestialBody.lastUpdate.tm_year
        hour = CelestialBody.lastUpdate.tm_hour
        hour = "{0:0=2d}".format(hour)
        minute = CelestialBody.lastUpdate.tm_min
        minute = "{0:0=2d}".format(minute)
        sec = CelestialBody.lastUpdate.tm_sec
        sec = "{0:0=2d}".format(sec)
        return "Updated: " + str(month)+"/"+str(day)+"/"+str(year)+" "+str(hour)+":"+str(minute)+":"+str(sec)

In [3]:
celestialBodies = []

In [24]:
def reloadBodies():
    celestialBodies = []
    #Opens website and dowloads html
    url = "https://theskylive.com/planets"
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()
    #HTML Parsing
    page_soup = soup(page_html, "html.parser")
    
    containers = page_soup.findAll("div", {"class":"object_container notvisible"})
    containers += page_soup.findAll("div", {"class":"object_container visible"})
    for container in containers:
        try:
            name = container["filter"]
        except:
            continue
        container_content = container.findAll("div", {"style":"text-align:center;vertical-align:center;"})
        info = container_content[0].text.replace('\n','')
        info = info.replace('\t','')
        info = info.replace('\xa0','')
        info = info.replace('\r','')
        info = info[0:info.index('Mag')]
        ra = info[info.index(':')+1 : info.index('s')+1]
        dec = info[info.index('Dec')+4:]
        celestialBodies.append(CelestialBody(name, ra, dec))
    CelestialBody.lastUpdate = time.localtime()
    return celestialBodies

In [25]:
celestialBodies = reloadBodies()
for body in celestialBodies:
    print(body.displayInfo())
CelestialBody.displayLastUpdate()

Name: the sun R.A: 08h 57m 34s Dec: +17° 12’ 28”
Name: jupiter R.A: 19h 24m 38s Dec: -22° 23’ 12”
Name: saturn R.A: 19h 57m 43s Dec: -20° 56’ 28”
Name: the moon R.A: 21h 22m 54s Dec: -20° 07’ 48”
Name: venus R.A: 05h 45m 30s Dec: +19° 31’ 43”
Name: mars R.A: 01h 17m 06s Dec: +03° 55’ 42”
Name: mercury R.A: 08h 00m 05s Dec: +21° 06’ 10”
Name: uranus R.A: 02h 32m 21s Dec: +14° 29’ 13”
Name: neptune R.A: 23h 25m 48s Dec: -04° 53’ 04”


'Updated: 8/3/2020 22:15:56'

## Converting Celestial Coordinates into Alt-Az

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

### Finding my IP adress by Web Scraping Google

In [37]:
chrome_options = Options()
chrome_options.add_argument("--headless")

PATH = '/Applications/chromedriver'
driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)
url = 'https://whatismyipaddress.com'
driver.get(url)
ip = ''
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "ipv4"))
    )
    ip = element.text
finally:
    driver.quit()

In [38]:
apiUrl = "http://api.ipstack.com/" +ip + "?access_key=0b762d0a0909e05254abb6c1278865c5"
apiInfo = urlopen(apiUrl)
data = json.load(apiInfo)
lat = data["latitude"]
print(lat)

29.740190505981445
